In [ ]:
from datasets import load_dataset

dataset = load_dataset("multi_news")
# dataset=dataset[:100]

In [ ]:
# dataset
train_data=dataset["train"]
validation_data= dataset["validation"]
test_data= dataset["test"]
# train_data=train_data[:100]
# print(train_data[0])

In [ ]:
import import_ipynb

In [ ]:
from rouge import Rouge
ROUGE = Rouge()

In [ ]:
from centroid import centre_bow
import os
from sklearn.metrics import pairwise_distances_argmin_min
import sklearn.ensemble as ensemble
import numpy as np
from preprocessing2 import Preprocessing
import operator
from sklearn.cluster import KMeans
from mmr import MMR
import argparse
from gensim.models import Word2Vec
from evaluate import load
import sklearn.preprocessing as preprocessing
import gensim
import sklearn.model_selection as ms
from gensim.models import Word2Vec


def summary_maker(input_sentences, best_selections, query, len, lambta, IDF):
    current_summary = [best_selections]
    sum_len = 0

    while (sum_len < len-1):
        MMRval = {}

        for i in input_sentences:
            MMRval[i] = MMR().get_mmr_score(i, query, current_summary, lambta, IDF)

        maximizer = max(MMRval, key=MMRval.get)
        current_summary.append(maximizer)
        input_sentences.remove(maximizer)

        if len(input_sentences) == 0:
            break
        sum_len += 1

    return current_summary

class Summarizer(object):
    def __init__(self, cluster_num=35, len=20):
        self.n_clusters = cluster_num
        self.len_summary = len

    @staticmethod
    def find_position(current_index, end_index):
        ranker = {}
        for i in current_index:
            for index, item in enumerate(end_index):
                if i < item:
                    ranker[i] = i - end_index[index - 1]
                    break
            if i not in ranker:
                ranker[i] = i - max(end_index)

        ranking_sorted = sorted(ranker.items(), key=operator.itemgetter(1))

        indexs = []
        for item in ranking_sorted:
            indexs.append(item[0])

        return indexs

    def kmean_summarizer(self, input_sentences, sent_text, org_sents, end_indx, cluster_num=30):
#         print(text_sents)
        model = Word2Vec(sent_text, min_count=1, vector_size=256, cbow_mean=1)
#         n_clusters=3
        X = []
        for current_sent in sent_text:
            temp = []
            word_counter = 0
            for current_word in current_sent:
                word_counter += 1
                if len(temp)==0:
                    temp = np.array(list(model.wv[current_word]))
                else:
                    temp += np.array(list(model.wv[current_word]))

            X.append(temp)
        
#         print(X)
        
#         print(len(X))
        cluster_num = min(len(X),cluster_num)
        
        my_kmeans = KMeans(cluster_num)
        my_kmeans = my_kmeans.fit(X)

        averages = []
        for k in range(cluster_num):
            my_index = np.where(my_kmeans.labels_ == k)[0]
            averages.append(np.mean(my_index))
        nearest, _ = pairwise_distances_argmin_min(my_kmeans.cluster_centers_, X)
        my_order = sorted(range(cluster_num), key=lambda k: averages[k])
        
#         print(ordering)
        
        curr_indices = [nearest[idx] for idx in my_order]
        pos_index = Summarizer.find_position(curr_indices, end_indx)

        new_sents = [org_sents[idx] for idx in pos_index]
        sen_obj = [input_sentences[idx] for idx in pos_index]

        return new_sents, sen_obj

    def summ_mmr(self, sen_obj, sent_indx, mode="train", sen_len=16):
        fresh_sentences = []

        for i in sent_indx:
            fresh_sentences.append(sen_obj[i])

        select_best = fresh_sentences[0]
        left_sents = fresh_sentences[1:]

        idfw = MMR.Inverse_Document_Freqs(fresh_sentences)

        if mode == "train":
            summ = summary_maker(left_sents, select_best, select_best, len(fresh_sentences) - 4, 0.4, idfw)
        else:
            summ = summary_maker(left_sents, select_best, select_best, sen_len, 0.4, idfw)

        final_summs = []
        for i in summ:
            final_summs.append(i.getOriginal_words())

        return final_summs

    def pos_add(self, sent_new, summs_mmr, sent_len, sent_indx, clust_n):
        output_indx = []
        for i, j in enumerate(sent_new):
            if j in summs_mmr:
                output_indx.append(i)

        add_num = sent_len - len(output_indx)

        add_indx = []
        for j in range(clust_n):
            if len(add_indx) == add_num:
                break

            if j not in sent_indx:
                add_indx.append(j)

        output_indx = sorted(output_indx + add_indx)

        return output_indx

    def centroid_summarizer(self, sen, sen_new, len=20):
        my_centr = centre_bow()

        sen_centrs = my_centr.start_summary(sen, sen_new, len)

        indxs_new = []
        for i in sen_centrs:
            for j, k in enumerate(sen):
                if i == k:
                    indxs_new.append(j)

        return indxs_new

    def get_summaries(self, sents, txts, sents_orgs, end_indxs, cent_len=20, mode="train"):
        clusters_num = self.n_clusters
        sent_len = self.len_summary
    
        new1, obj_sents = self.kmean_summarizer(sents, txts, sents_orgs, end_indxs, clusters_num)

        inputs = []

        for i in obj_sents:
            inputs.append(i.getOriginal_words())

        sent_indxs = self.centroid_summarizer(inputs, new1, cent_len)

        summs_from_mmr = self.summ_mmr(obj_sents, sent_indxs, mode)

        output_indxs = self.pos_add(new1, summs_from_mmr, sent_len, sent_indxs, clusters_num)
            
#         final_index = self.add_position(new_sents, mmr_summaries, len_sent, index_sents, n_clusters)

#         print(n_clusters)
#         print(final_index)
#         print(len(new_sents))
        
        output_summaries = []
        for i in output_indxs:
            output_summaries.append(new1[i])

#         for item in index_sents:
#               final_summaries.append(new_sents[item])

    
        return output_summaries

if __name__ == "__main__":

    check_count=0
    get_answers=[]
    for current in train_data:
        check_count=check_count+1
        print("Iteration No------>",check_count)
        if check_count>100:
            break

        sents, end_indxs = Preprocessing().directory_opener(current)

        sent_txts = []
        for i in sents:
            sent_txts.append(i.get_stemmed_words())

        sentences_clean = []
        sentences_org = []
        for i in sents:
            sentences_org.append(i.getOriginal_words())

            temp_var = ""
            for j in i.get_stemmed_words():
                temp_var += j + " "

            if temp_var[-1] not in sentences_clean:
                sentences_clean.append(temp_var[:-1])

        this_summarizer = Summarizer(cluster_num=5
#         summarizer = Summarizer(n_clusters=20
                            , len=20)
        
        print("Given Summary------",current["summary"])
        final_output = this_summarizer.get_summaries(sents, sent_txts, sentences_org, end_indxs)

#         with open(path_to_save + folder + ".me", 'w') as fileOut:
#             fileOut.write("\n".join(summary))
        print("Predicted Summary------",final_output[0])
#         Rouge_score=ROUGE.get_scores(folder["summary"], summary[0])
        bertscore = load("bertscore")
#         predictions = ["hello there"]
        references = [current["summary"]]
        results = bertscore.compute(predictions=final_output, references=references, lang="en")
        print("bertscore----->",results)
        get_answers.append(results['f1'][0])
        print(np.mean(get_answers))
print("Mean_F1----->",np.mean(get_answers))        
#         print("Rouge Score----->",Rouge_score)

In [ ]:
fig, axs = plt.subplots(1, 1,
                        figsize =(10, 7),
                        tight_layout = True)
plt.xlabel("F1 Score using BERT",size=18)
plt.ylabel("No. of Inputs with given F1 score",size=18)

 
axs.hist(get_answers, bins = 10)
 